# Monthly averaging dev

In [2]:
# Basic packages
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import math


# DateTime packages
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
import time
import matplotlib.dates as mdates

# Stats packages
import scipy
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import seaborn as sns
import pyleoclim as pyleo
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Logistical packages
import requests
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

In [31]:
def make_month_lims(M, leapyear = None):
    day_i = 1
    if M == 1 or M == 3 or M ==5 or M == 7 or M == 8 or M == 10 or M == 12:
        day_f = 31
    elif M == 2:
        if leapyear == True:
            day_f = 29
        else:
            day_f = 28
    else:
        day_f = 30
    return day_i, day_f

In [77]:
df = xr.open_mfdataset('/Volumes/Baskind/ROMS/April 2024/apr2024_2005/ocean_his_0001.nc').mean('ocean_time')
JAN = xr.full_like(df, np.nan).assign_coords(time=np.nan)
FEB = xr.full_like(df, np.nan).assign_coords(time=np.nan)
MAR = xr.full_like(df, np.nan).assign_coords(time=np.nan)
APR = xr.full_like(df, np.nan).assign_coords(time=np.nan)
MAY = xr.full_like(df, np.nan).assign_coords(time=np.nan)
JUN = xr.full_like(df, np.nan).assign_coords(time=np.nan)
JUL = xr.full_like(df, np.nan).assign_coords(time=np.nan)
AUG = xr.full_like(df, np.nan).assign_coords(time=np.nan)
SEP = xr.full_like(df, np.nan).assign_coords(time=np.nan)
OCT = xr.full_like(df, np.nan).assign_coords(time=np.nan)
NOV = xr.full_like(df, np.nan).assign_coords(time=np.nan)
DEC = xr.full_like(df, np.nan).assign_coords(time=np.nan)

PATH = '/Volumes/Baskind/ROMS/April 2024/apr2024_2005/'
filebase = 'ocean_his_'
fileext = '.nc'
YEAR = 2005
LEAPYEAR = False

nrange = np.arange(1,10,1)
for m in nrange:
    N = str(m).rjust(4, "0")
    FILE = PATH + filebase + N + fileext
    filename = filebase + N + fileext
    df = xr.open_mfdataset(FILE)
    print('Working on Month #' + str(df["ocean_time"].max().dt.month.max().values) + ' in ' + filename + '...')
    
    # ds = df.where(df["ocean_time"] < np.datetime64(datetime(2005,1,31,23,59)))
    # t = ds['ocean_time'].mean()
    # ds = ds.mean('ocean_time')
    # ds = ds.assign_coords(time=t)
    # dftemp = xr.concat([ds,dftemp], dim='time')
    
    # If working dataset only has values from a single month
    # i.e. max month equals min month of the dataset
    if df["ocean_time"].min().dt.month == df["ocean_time"].max().dt.month:
        # Taking mean of month to reduce array to 1 value
        MONTH = df["ocean_time"].dt.month.mean()
    
        # # Set day limits for the month
        # # e.g. October has days 1-31 but April has days 1-30
        # dayi, dayf = make_month_lims(MONTH)
        # ti = np.datetime64(datetime(2005,MONTH,dayi,0,0))
        # tf = np.datetime64(datetime(2005,MONTH,dayf,23,59))
        # # Time slice
        # ds = df.where(df["ocean_time"] >= ti).where(df["ocean_time"] < tf)
        ds = df
    
        # Get mean "filler" time
        t = ds['ocean_time'].mean()
        # Time mean
        ds = ds.mean('ocean_time')
        # Assign a time dimension with mean filler time
        ds = ds.assign_coords(time=t)
        if len(ds.keys()) > 200:
            ds = ds.drop_vars(["dstart"])
        
        del df
    
        # FOR EACH MONTH
        # Select the appropriate dataset for the month (e.g. JAN for January)
        # Concatenate working dataset with month dataset
        # Take the mean time for the month dataset as a filler time
        # Take time mean of month dataset and assign a time dim with filler time
        if MONTH == 1:                                             # JANUARY
            JAN = xr.concat([ds,JAN], dim='time')
            T = JAN['time'].mean()
            JAN = JAN.mean('time').assign_coords(time = T)
        elif MONTH == 2:                                           # FEBRUARY
            FEB = xr.concat([ds,FEB], dim='time')
            T = FEB['time'].mean()
            FEB = FEB.mean('time').assign_coords(time = T)
        elif MONTH == 3:                                           # MARCH
            MAR = xr.concat([ds,MAR], dim='time')
            T = MAR['time'].mean()
            MAR = MAR.mean('time').assign_coords(time = T)
        elif MONTH == 4:                                           # APRIL
            APR = xr.concat([ds,APR], dim='time')
            T = APR['time'].mean()
            APR = APR.mean('time').assign_coords(time = T)
        elif MONTH == 5:                                           # MAY
            MAY = xr.concat([ds,MAY], dim='time')
            T = MAY['time'].mean()
            MAY = MAY.mean('time').assign_coords(time = T)
        elif MONTH == 6:                                           # JUNE
            JUN = xr.concat([ds,JUN], dim='time')
            T = JUN['time'].mean()
            JUN = JUN.mean('time').assign_coords(time = T)
        elif MONTH == 7:                                           # JULY
            JUL = xr.concat([ds,JUL], dim='time')
            T = JUL['time'].mean()
            JUL = JUL.mean('time').assign_coords(time = T)
        elif MONTH == 8:                                           # AUGUST
            AUG = xr.concat([ds,AUG], dim='time')
            T = AUG['time'].mean()
            AUG = AUG.mean('time').assign_coords(time = T)
        elif MONTH == 9:                                           # SEPTEMBER
            SEP = xr.concat([ds,SEP], dim='time')
            T = SEP['time'].mean()
            SEP = SEP.mean('time').assign_coords(time = T)
        elif MONTH == 10:                                          # OCTOBER
            OCT = xr.concat([ds,OCT], dim='time')
            T = OCT['time'].mean()
            OCT = OCT.mean('time').assign_coords(time = T)
        elif MONTH == 11:                                          # NOVEMBER
            NOV = xr.concat([ds,NOV], dim='time')
            T = NOV['time'].mean()
            NOV = NOV.mean('time').assign_coords(time = T)
        elif MONTH == 12:                                          # DECEMBER
            DEC = xr.concat([ds,DEC], dim='time')
            T = DEC['time'].mean()
            DEC = DEC.mean('time').assign_coords(time = T)
        del ds
    # =======================================================================================================================
    # =======================================================================================================================
    
    # If working dataset does NOT has values from a single month
    # i.e. max month does NOT equal min month of the dataset
    else:
        # Start with first (min) month ****************************************
        MONTH1 = df["ocean_time"].dt.month.min()
    
        # Set day limits for the first month
        # e.g. October has days 1-31 but April has days 1-30
        dayi1, dayf1 = make_month_lims(MONTH1, leapyear = LEAPYEAR)
        ti1 = np.datetime64(datetime(YEAR,MONTH1,dayi1,0,0))
        tf1 = np.datetime64(datetime(YEAR,MONTH1,dayf1,23,59))
    
        # Time slice
        ds1 = df.where(df["ocean_time"] >= ti1).where(df["ocean_time"] < tf1)
        # Get mean "filler" time
        t1 = ds1['ocean_time'].mean()
    
        # Time mean
        ds1 = ds1.mean('ocean_time')
        # Assign a time dimension with mean filler time
        ds1 = ds1.assign_coords(time=t1)
        if len(ds1.keys()) > 200:
            ds1 = ds1.drop_vars(["dstart"])
    
        # FOR EACH MONTH
        # Select the appropriate dataset for the month (e.g. JAN for January)
        # Concatenate working dataset with month dataset
        # Take the mean time for the month dataset as a filler time
        # Take time mean of month dataset and assign a time dim with filler time
        if MONTH1 == 1:                                             # JANUARY
            JAN = xr.concat([ds1,JAN], dim='time')
            T = JAN['time'].mean()
            JAN = JAN.mean('time').assign_coords(time = T)
        elif MONTH1 == 2:                                           # FEBRUARY
            FEB = xr.concat([ds1,FEB], dim='time')
            T = FEB['time'].mean()
            FEB = FEB.mean('time').assign_coords(time = T)
        elif MONTH1 == 3:                                           # MARCH
            MAR = xr.concat([ds1,MAR], dim='time')
            T = MAR['time'].mean()
            MAR = MAR.mean('time').assign_coords(time = T)
        elif MONTH1 == 4:                                           # APRIL
            APR = xr.concat([ds1,APR], dim='time')
            T = APR['time'].mean()
            APR = APR.mean('time').assign_coords(time = T)
        elif MONTH1 == 5:                                           # MAY
            MAY = xr.concat([ds1,MAY], dim='time')
            T = MAY['time'].mean()
            MAY = MAY.mean('time').assign_coords(time = T)
        elif MONTH1 == 6:                                           # JUNE
            JUN = xr.concat([ds1,JUN], dim='time')
            T = JUN['time'].mean()
            JUN = JUN.mean('time').assign_coords(time = T)
        elif MONTH1 == 7:                                           # JULY
            JUL = xr.concat([ds1,JUL], dim='time')
            T = JUL['time'].mean()
            JUL = JUL.mean('time').assign_coords(time = T)
        elif MONTH1 == 8:                                           # AUGUST
            AUG = xr.concat([ds1,AUG], dim='time')
            T = AUG['time'].mean()
            AUG = AUG.mean('time').assign_coords(time = T)
        elif MONTH1 == 9:                                           # SEPTEMBER
            SEP = xr.concat([ds1,SEP], dim='time')
            T = SEP['time'].mean()
            SEP = SEP.mean('time').assign_coords(time = T)
        elif MONTH1 == 10:                                          # OCTOBER
            OCT = xr.concat([ds1,OCT], dim='time')
            T = OCT['time'].mean()
            OCT = OCT.mean('time').assign_coords(time = T)
        elif MONTH1 == 11:                                          # NOVEMBER
            NOV = xr.concat([ds1,NOV], dim='time')
            T = NOV['time'].mean()
            NOV = NOV.mean('time').assign_coords(time = T)
        elif MONTH1 == 12:                                          # DECEMBER
            DEC = xr.concat([ds1,DEC], dim='time')
            T = DEC['time'].mean()
            DEC = DEC.mean('time').assign_coords(time = T)
    
        # Go to next (max) month ***********************************************
        MONTH2 = df["ocean_time"].dt.month.max()
    
        # Set day limits for the first month
        # e.g. October has days 1-31 but April has days 1-30
        dayi2, dayf2 = make_month_lims(MONTH2, leapyear = LEAPYEAR)
        ti2 = np.datetime64(datetime(YEAR,MONTH2,dayi2,0,0))
        tf2 = np.datetime64(datetime(YEAR,MONTH2,dayf2,23,59))
    
        # Time slice
        ds2 = df.where(df["ocean_time"] >= ti2).where(df["ocean_time"] < tf2)
        # Get mean "filler" time
        t2 = ds2['ocean_time'].mean()
    
        # Time mean
        ds2 = ds2.mean('ocean_time')
        # Assign a time dimension with mean filler time
        ds2 = ds2.assign_coords(time=t2)
        if len(ds2.keys()) > 200:
            ds2 = ds2.drop_vars(["dstart"])
    
        # FOR EACH MONTH
        # Select the appropriate dataset for the month (e.g. JAN for January)
        # Concatenate working dataset with month dataset
        # Take the mean time for the month dataset as a filler time
        # Take time mean of month dataset and assign a time dim with filler time
        if MONTH2 == 1:                                             # JANUARY
            JAN = xr.concat([ds2,JAN], dim='time')
            T = JAN['time'].mean()
            JAN = JAN.mean('time').assign_coords(time = T)
        elif MONTH2 == 2:                                           # FEBRUARY
            FEB = xr.concat([ds2,FEB], dim='time')
            T = FEB['time'].mean()
            FEB = FEB.mean('time').assign_coords(time = T)
        elif MONTH2 == 3:                                           # MARCH
            MAR = xr.concat([ds2,MAR], dim='time')
            T = MAR['time'].mean()
            MAR = MAR.mean('time').assign_coords(time = T)
        elif MONTH2 == 4:                                           # ARPIL
            APR = xr.concat([ds2,APR], dim='time')
            T = APR['time'].mean()
            APR = APR.mean('time').assign_coords(time = T)
        elif MONTH2 == 5:                                           # MAY
            MAY = xr.concat([ds2,MAY], dim='time')
            T = MAY['time'].mean()
            MAY = MAY.mean('time').assign_coords(time = T)
        elif MONTH2 == 6:                                           # JUNE
            JUN = xr.concat([ds2,JUN], dim='time')
            T = JUN['time'].mean()
            JUN = JUN.mean('time').assign_coords(time = T)
        elif MONTH2 == 7:                                           # JULY
            JUL = xr.concat([ds2,JUL], dim='time')
            T = JUL['time'].mean()
            JUL = JUL.mean('time').assign_coords(time = T)
        elif MONTH2 == 8:                                           # AUGUST
            AUG = xr.concat([ds2,AUG], dim='time')
            T = AUG['time'].mean()
            AUG = AUG.mean('time').assign_coords(time = T)
        elif MONTH2 == 9:                                           # SEPTEMBER
            SEP = xr.concat([ds2,SEP], dim='time')
            T = SEP['time'].mean()
            SEP = SEP.mean('time').assign_coords(time = T)
        elif MONTH2 == 10:                                          # OCTOBER
            OCT = xr.concat([ds2,OCT], dim='time')
            T = OCT['time'].mean()
            OCT = OCT.mean('time').assign_coords(time = T)
        elif MONTH2 == 11:                                          # NOVEMBER
            NOV = xr.concat([ds2,NOV], dim='time')
            T = NOV['time'].mean()
            NOV = NOV.mean('time').assign_coords(time = T)
        elif MONTH2 == 12:                                          # DECEMBER
            DEC = xr.concat([ds2,DEC], dim='time')
            T = DEC['time'].mean()
            DEC = DEC.mean('time').assign_coords(time = T)
        del ds1, ds2
    

Working on Month #1 in ocean_his_0001.nc...
Working on Month #1 in ocean_his_0002.nc...
Working on Month #1 in ocean_his_0003.nc...
Working on Month #1 in ocean_his_0004.nc...
Working on Month #1 in ocean_his_0005.nc...
Working on Month #1 in ocean_his_0006.nc...
Working on Month #2 in ocean_his_0007.nc...
Working on Month #2 in ocean_his_0008.nc...
Working on Month #2 in ocean_his_0009.nc...
